In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('stores_with_addresses.csv')

/opt/anaconda3/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.tail()

,Store Format,Address,Brand,2018 / January,2018 / February,2018 / March,2018 / April,2018 / May,2018 / June,2018 / July,...,2019 / March,2019 / April,2019 / May,2019 / June,2019 / July,2019 / August,2019 / September,2019 / October,2019 / November,2019 / December
156044,NaN,NaN,Organika,"0,1","0,3","0,3",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,1"
156045,NaN,NaN,Spelta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"0,1",NaN,NaN,NaN,NaN
156046,NaN,NaN,Vodka Other Brand,"0,2","0,3","0,3",NaN,NaN,NaN,NaN,...,"0,2","0,2","0,4","0,3","0,2","0,1","0,5",NaN,"0,4","0,1"
156047,NaN,"Россия, г Москва, ш Ярославское, д 12 Total",NaN,"15,4","15,0","15,0",NaN,NaN,NaN,NaN,...,"11,8","9,1","9,8","11,0","10,6","11,5","9,2","8,0","8,3","12,6"
156048,Supermarket Total,NaN,NaN,"54 120,1","57 690,5","60 223,8","52 646,9","48 035,1","46 917,6","44 978,7",...,"54 341,2","48 476,7","45 288,3","42 107,9","46 270,5","49 884,4","49 344,1","54 532,9","58 291,0","69 117,5"


In [4]:
month_columns = [(c, pd.datetime.strptime(c, ' %Y / %B ')) for c in df.columns[3:]]
new_rows = []

store_format = None
address = None
skipped = 0
for index, row in df.iterrows():
    store_format = row['Store Format'] if not pd.isnull(row['Store Format']) else store_format
    if not store_format:
        raise ValueError('No store format!')
    address = row['Address'] if not pd.isnull(row['Address']) else address
    if not address:
        raise ValueError('No address!')
    if 'Total' in address:
        skipped += 1
        continue
    for col_name, col_date in month_columns:
        row_dict = dict(
            brand=row['Brand'], 
            store_format=store_format, 
            address=address,
            month=col_date.month,
            year=col_date.year,
            sales=row[col_name],
        )
        new_rows.append(row_dict)

new_df = pd.DataFrame(new_rows, columns=['year', 'month', 'store_format', 'address', 'brand', 'sales'])  

In [5]:
new_df.tail()

,year,month,store_format,address,brand,sales
3687883,2019,8,Supermarket,"Россия, г Москва, ш Ярославское, д 12",Vodka Other Brand,"0,1"
3687884,2019,9,Supermarket,"Россия, г Москва, ш Ярославское, д 12",Vodka Other Brand,"0,5"
3687885,2019,10,Supermarket,"Россия, г Москва, ш Ярославское, д 12",Vodka Other Brand,NaN
3687886,2019,11,Supermarket,"Россия, г Москва, ш Ярославское, д 12",Vodka Other Brand,"0,4"
3687887,2019,12,Supermarket,"Россия, г Москва, ш Ярославское, д 12",Vodka Other Brand,"0,1"


In [6]:
(len(df) - skipped) * 24 == (len(new_df))

True

In [7]:
len(new_df)

3687888

In [8]:
len(new_df[new_df['address'].isin(
    ['Россия, г Москва', 
     'Россия, Московская обл, Сергиево-Посадский р-н, снт Москва (д Ильинки), д 17',
     'Россия, Московская обл, Сергиево-Посадский р-н, снт Москва (д Ильинки)',]
    )])

3720

In [9]:
clean_new_df = new_df.drop(new_df[new_df['address'].isin(
    ['Россия, г Москва', 
     'Россия, Московская обл, Сергиево-Посадский р-н, снт Москва (д Ильинки), д 17',
     'Россия, Московская обл, Сергиево-Посадский р-н, снт Москва (д Ильинки)',]
    )]['address'].index)

In [10]:
len(clean_new_df)

3684168

In [11]:
clean_new_df = clean_new_df.sort_values(by=['year', 'month'])

In [12]:
def transorm_sales(sales_value):
    try:
        return float(str(sales_value).strip().replace(',', '.'))
    except ValueError:
        return 0

In [13]:
clean_new_df['sales'] = clean_new_df['sales'].apply(transorm_sales).astype(np.float32)

In [14]:
clean_new_df.to_csv('sales_transformed.csv')